# Scraper para variables

In [11]:
# TODAS LAS LIBRERÍAS PARA SCRAPEAR
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.alert import Alert
import time
import pandas as pd
import shutil
from datetime import datetime
from selenium.webdriver.common.action_chains import ActionChains
import requests
import PyPDF2
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

## Scrapper cédula profesional

### Conexión al navegador

In [ ]:
chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)
start_url = 'https://cedulaprofesionalsep.online/#Consulta_de_Cedula_Profesional'
driver.get(start_url)
# Modificar el tamaño de la ventana
#driver.execute_script("document.body.style.zoom='50%'")

> Función para obtener el tamaño de una solicitud HTTP

In [30]:
def get_request_size(url):
    response = requests.get(url)
    return len(response.content)

### Leer archivo CSV y obtener lista de nombres

In [7]:
init=2
final=500

df = pd.read_excel('Z:/Data/HSBC/Asignaciones/ASG_TDC_IA_ENERO_JUNIO.xlsx')
df['Nombre'] = df.Nombre.apply(lambda X: X.replace('/', ' ').replace('*', '').rstrip())
df_filtered = df['Nombre']
df_filtered.drop_duplicates(inplace=True)
df_input = pd.DataFrame(df_filtered.reset_index(drop=True)).iloc[init:final]



df_input['NombreSplit'] = df_input.Nombre.apply(lambda X: split_name(X))
df_input['Nombre'] = df_input.NombreSplit.apply(lambda X: X[0])
df_input['Paterno'] = df_input.NombreSplit.apply(lambda X: X[1])
df_input['Materno'] = df_input.NombreSplit.apply(lambda X: X[2])

# Concatenación
df_input['Nombre_Completo'] = df_input.apply(lambda row: ' '.join([row['Nombre'], row['Paterno'], row['Materno']]), axis = 1)
# Cambiar nombre
#df.rename(columns={'Nombre':'Nombre'}, inplace=True)
df_input.tail()

TypeError: sequence item 2: expected str instance, NoneType found

In [4]:
df_input.head()

,Nombre,NombreSplit,Paterno,Materno,Nombre_Completo
192,jose de jesus,"[jose de jesus, hernandez, hernandez]",hernandez,hernandez,jose de jesus hernandez hernandez
193,jose adan,"[jose adan, perez, tamayo]",perez,tamayo,jose adan perez tamayo
194,alejandro,"[alejandro, perez, alcantara]",perez,alcantara,alejandro perez alcantara
195,j ascencion,"[j ascencion, pina, patino]",pina,patino,j ascencion pina patino
196,miguel angel,"[miguel angel, sanchez, ramos]",sanchez,ramos,miguel angel sanchez ramos


### Scrapper

In [5]:
# Código para modificar la ventana
driver.execute_script("window.scrollTo(10, 7600);")

In [ ]:
data=[]
#total_data_downloaded = 0
for index, row in df_input.iterrows():
    nombre_persona = row['Nombre_Completo']
   
    # Se escrolea hasta la parte específica 
    driver.execute_script("window.scrollTo(10, 7600);")
    time.sleep(5)
    
    # Se ingresan las variables del scrapeo
    nombre=row['Nombre']
    paterno=row['Paterno']
    materno=row['Materno']
    
    # Ingresar nombres
    celda_nombre = driver.find_element(By.XPATH, '//*[@id="input-nombre"]')
    celda_nombre.clear()
    celda_nombre.send_keys(str(nombre))

    #Ingresar Apellido Paterno
    celda_paterno = driver.find_element(By.XPATH, '//*[@id="input-apaterno"]')
    celda_paterno.clear()
    celda_paterno.send_keys(str(paterno))

    # Ingresar Apellido Materno
    celda_materno = driver.find_element(By.XPATH, '//*[@id="input-amaterno"]')
    celda_materno.clear()
    celda_materno.send_keys(str(materno))

    # Click en botón buscar
    buscar = driver.find_element(By.XPATH, '//*[@id="container-form-1"]/form/div[4]/div/input')
    buscar.click()
    time.sleep(2)

    max_wait_time = 5  # Tiempo máximo de espera en segundos
    start_time = time.time()  # Momento de inicio de espera

    while time.time() - start_time < max_wait_time:
        try:
            alert = driver.switch_to.alert
            alert.accept()
            driver.refresh()
            break  # Salir del bucle si se encuentra la ventana emergente
        except:

            time.sleep(1)  # Esperar 1 segundo antes de intentar nuevamente
            
            


    time.sleep(5)
    
    # Esperar a que los elementos estén visibles
    ver_botones = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'ver-mas')))

    
    for ver in ver_botones:
        nombre_boton = ver.get_attribute("value")
        print(nombre_boton)
        if unidecode(nombre_boton.lower()) == unidecode(nombre_persona.lower()):
            cantidad_registros = len(ver_botones)

            ver.click()
            
            # Encontrar los elementos de entrada de texto por su id
            input_cedula = driver.find_element(By.ID, "input-cedula-result")
            input_tipo = driver.find_element(By.ID, "input-tipo-result")
            input_sexo = driver.find_element(By.ID, "input-sexo-result")
            input_nombre = driver.find_element(By.ID, "input-nombre-result")
            input_paterno = driver.find_element(By.ID, "input-paterno-result")
            input_materno = driver.find_element(By.ID, "input-materno-result")
            input_escuela = driver.find_element(By.ID, "input-escuela-result")
            input_titulo = driver.find_element(By.ID, "input-titulo-result")

            # Obtener los valores de los campos
            valor_cedula = input_cedula.get_attribute("value")
            valor_tipo = input_tipo.get_attribute("value")
            valor_sexo = input_sexo.get_attribute("value")
            valor_nombre = input_nombre.get_attribute("value")
            valor_paterno = input_paterno.get_attribute("value")
            valor_materno = input_materno.get_attribute("value")
            valor_escuela = input_escuela.get_attribute("value")
            valor_titulo = input_titulo.get_attribute("value")

            # Agregar los valores a la lista de diccionarios
            data.append({
            "Cédula": valor_cedula,
            "Tipo de Cédula": valor_tipo,
            "Sexo": valor_sexo,
            "Nombre": valor_nombre,
            "Paterno": valor_paterno,
            "Materno": valor_materno,
            "Escuela": valor_escuela,
            "Título": valor_titulo
            })
            # Encontrar los elementos de entrada de texto por su id
            cerrar = driver.find_element(By.XPATH, '//*[@id="cerrar_resut_personal"]')
            cerrar.click()
       
        
        # Obtener el tamaño de la página descargada
        #page_source_size = len(driver.page_source.encode('utf-8'))
        #total_data_downloaded += page_source_size
        
driver.quit()            
df_data = pd.DataFrame(data)
df_data.to_csv('Z:/Data/HSBC/SCRAPPING/Cédulas/0_500_.csv')
    

                



In [ ]:
df_data.tail(10)

In [37]:
nombre_buscado = "jose de jesus"
resultado = df_input.loc[df_input['Nombre'] == nombre_buscado]
resultado

,Nombre,NombreSplit,Paterno,Materno
192,jose de jesus,"[jose de jesus, hernandez, hernandez]",hernandez,hernandez


In [39]:
# Convertir los datos en un DataFrame y guardar en CSV              
df_data = pd.DataFrame(data)
df_data.to_csv('Z:/Data/HSBC/SCRAPPING/Cédulas/0_192_.csv')
    

## IFT

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import zipfile
import os
from datetime import datetime
from selenium.webdriver.common.action_chains import ActionChains
import warnings
warnings.filterwarnings('ignore')

def ift_automation():
    # Configura las opciones de Chrome para la descarga
    #download_folder = "/home/bryan/Documentos/airflow/outputs/ift"
    chrome_options = Options()
    #chrome_options.add_argument('--headless')
    chrome_options.add_experimental_option("prefs", {
        "download.prompt_for_download": False,
        #"download.default_directory": download_folder,  # Utiliza la ubicación actual del script como carpeta de descarga
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True, # Desactiva la verificación de seguridad de descargas
        "profile.default_content_settings.popups":0,
        "profile.default_content_setting_values.automatic_downloads": 2,
    })

    # Configuración para ingresar al explorador
    remote_webdriver = 'remote_chromedriver'
    driver = webdriver.Chrome( options=chrome_options)
    url = "https://sns.ift.org.mx:8081/sns-frontend/planes-numeracion/descarga-publica.xhtml"
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    time.sleep(10)
    button = wait.until(EC.element_to_be_clickable((By.ID, "FORM_planes:BTN_planPublico1")))
    button.send_keys(Keys.ENTER)

    # Espera un tiempo para que la descarga se complete
    tiempo_espera = 60  # Puedes ajustar este valor según sea necesario
    tiempo_transcurrido = 0
    archivo_zip = None

    while tiempo_transcurrido < tiempo_espera:
        archivos_en_carpeta = os.listdir("/opt/airflow/outputs/ift/")
        for archivo in archivos_en_carpeta:
            if archivo.endswith('.zip'):
                archivo_zip = archivo
                break

        if archivo_zip:
            break

        time.sleep(1)
        tiempo_transcurrido += 1

    if archivo_zip:
        # Abre el archivo ZIP en modo de lectura
        with zipfile.ZipFile(os.path.join("/opt/airflow/outputs/ift/", archivo_zip), 'r') as zf:
            # Extrae todos los archivos y carpetas dentro del archivo ZIP en la carpeta de destino
            zf.extractall("/opt/airflow/outputs/ift/")

    driver.quit()
ift_automation()

FileNotFoundError: [Errno 2] No such file or directory: '/opt/airflow/outputs/ift/'

## Scraper marcatel

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
from datetime import datetime, timedelta, date
from dotenv import load_dotenv, find_dotenv
import pandas as pd
import json
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import warnings
warnings.filterwarnings('ignore')

def marcatel_automation():
    url = "https://tink.marcatel.com.mx/Login.aspx"
    
    load_dotenv(find_dotenv()) # Load the .env file.
    userid = os.getenv("USER_MARCATEL")
    passwo = os.getenv("PASSWORD_MARCATEL")
    
   
    chrome_options = Options()
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    user = wait.until(EC.presence_of_element_located((By.ID, "usuario")))
    user.click()
    user.send_keys(userid)
    contra = wait.until(EC.presence_of_element_located((By.ID, "contrasena")))
    
        # Pasar credenciales
    contra.click()
    contra.send_keys(passwo)
    init_button = wait.until(EC.element_to_be_clickable((By.ID, "btnini")))
    init_button.click()
    time.sleep(10)
        # Abrir ventana de reportes SMS
        # Redirecciona a la URL deseada
    driver.get("https://tink.marcatel.com.mx/Reportes/wfReporteRespuestas.aspx")
        # Espera unos segundos antes de cerrar el navegador
    time.sleep(10)
    
    # Get current date and time
    current_date = datetime.now()
    
    # Yesterday's date
    yersterday_date = current_date - timedelta(days=30)
    
    # Format date
    formatted_date = yersterday_date.strftime("%Y-%m-%d")
    print(formatted_date)
    # Date to marcatel
    date_marcatel = wait.until(EC.presence_of_element_located((By.ID, 'start')))
    date_marcatel.click()
    date_marcatel.clear()
    date_marcatel.send_keys(formatted_date)
    
    time.sleep(6)

    reporte = wait.until(EC.element_to_be_clickable((By.ID, "generarReporte")))
    reporte.click()
    time.sleep(30)
    exportar = driver.find_element(By.ID, "excel")
    driver.execute_script("arguments[0].scrollIntoView();", exportar)
    exportar.click()
    time.sleep(10)
    driver.quit()
marcatel_automation()


2023-09-10


: 

## Scraper Partidos Políticos y gobernadores

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import csv
import time
import re
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
import os
import pandas as pd
import json
import warnings
warnings.filterwarnings('ignore')


def gobernadores_scraper():
    url = 'https://www.conago.org.mx/gobernadores'
    

   
    # Configuración para evitar notificaciones
    chrome_options = Options()
    #options.add_argument("window-size=1200x600")
    chrome_options.add_experimental_option("prefs", {
    "download.prompt_for_download": False,  # Desactiva la ventana emergente de descarga
    "download.directory_upgrade": True,
    "safebrowsing.enabled": False,  # Desactiva la verificación de seguridad de descargas
    #"profile.default_content_settings.popups":0,
    "download.default_directory":"/opt/airflow/outputs/Gobernadores/"})
    # Configuración para ingresar al explorador

    remote_webdriver = 'remote_chromedriver'
    driver = webdriver.Chrome( options=chrome_options)
    driver.get(url)

    # Localiza los elementos de interés
    elementos = driver.find_elements(By.CLASS_NAME, 'col-xs-12.col-md-8.texto.rcGobernadores')


    # Crear o abrir el archivo CSV
    with open('datos.csv', 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Nombre', 'Fecha de Inicio', 'Fecha de Fin', 'Estado']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()  # Escribir la fila de encabezados
        
        for _ in range(len(elementos)):
            # Obtén el elemento específico y el estado
            elemento = elementos[_]
            estado_element = elemento.find_element(By.CLASS_NAME, 'textMedio')
            estado = estado_element.text
            estado_element.click()  # Hacer clic en el enlace
            
            # Obtener los elementos de interés en la página de detalle del estado
            elementos_estado = driver.find_elements(By.CLASS_NAME, 'col-xs-12.col-sm-6.col-md-4.pb15')

            for elemento_estado in elementos_estado:
                nombre_gobernador = elemento_estado.find_element(By.TAG_NAME, 'h4').text
                html = elemento_estado.get_attribute('innerHTML')
                
                # Usar expresión regular para encontrar las fechas
                fechas = re.search(r'(\d{2}/\d{2}/\d{4}) a (\d{2}/\d{2}/\d{4})', html)
                
                if fechas:
                    fecha_inicio = fechas.group(1)
                    fecha_fin = fechas.group(2)
                else:
                    fecha_inicio = "Fecha no disponible"
                    fecha_fin = "Fecha no disponible"
                
                # Imprimir para verificar
                print(nombre_gobernador, fecha_inicio, fecha_fin)
                
                # Escribir los datos en el archivo CSV
                writer.writerow({'Nombre': nombre_gobernador, 'Fecha de Inicio': fecha_inicio,
                                'Fecha de Fin': fecha_fin, 'Estado': estado})

            driver.back()  # Regresar a la página anterior

        # Cerrar el navegador
        driver.quit()
gobernadores_scraper()



def partidos_scraper():
    # Configura el controlador de Chrome
    driver = webdriver.Chrome()

    # Abre la página web de Wikipedia
    url = 'https://es.wikipedia.org/wiki/Wikipedia:Portada'
    driver.get(url)

    # Leer los nombres de los gobernadores desde el archivo CSV
    nombres_gobernadores = []
    resultados = []

    with open('datos.csv', 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            nombre_completo = row['Nombre']
            
            # Buscar el último punto en el nombre
            ultimo_punto_index = nombre_completo.rfind('.')
            if ultimo_punto_index != -1:
                nombre_completo = nombre_completo[ultimo_punto_index+1:].strip()
            
            nombres_gobernadores.append(nombre_completo)

    # Iterar sobre los nombres y buscar en la página web
    for nombre in nombres_gobernadores:
        buscar_gobernador = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'cdx-text-input__input'))
        )

        # Usar ActionChains para borrar y escribir
        actions = ActionChains(driver)
        actions.click(buscar_gobernador).key_down(Keys.CONTROL).send_keys('a').key_up(Keys.CONTROL).send_keys(Keys.DELETE).send_keys(nombre).perform()
        buscar_gobernador = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'cdx-text-input__input'))
        )
        time.sleep(5)
        buscar_gobernador.send_keys(Keys.ENTER)
        #button_search = WebDriverWait(driver, 10).until((EC.element_to_be_clickable((By.CLASS_NAME, 'cdx-button cdx-button--action-default cdx-button--weight-normal cdx-button--size-medium cdx-button--framed cdx-search-input__end-button'))))
        #button_search.click()
        #buscar_gobernador.submit()
        buscar_gobernador = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'cdx-text-input__input'))
        )
        # Esperar a que se cargue la página del artículo
        driver.implicitly_wait(10)

        # Verificar si la página muestra resultados de búsqueda
        try:
            no_results_heading = driver.find_element(By.CLASS_NAME, 'firstHeading.mw-first-heading')
            if no_results_heading.text == 'Resultados de la búsqueda':
                print(f"No se encontró información para {nombre}")
                print("---")
                continue
        except NoSuchElementException:
            pass

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        infobox = soup.find('table', {'class': 'infobox biography vcard'})

        if infobox:
            partido_element = infobox.find('th', string='Partido político')
            if partido_element:
                partido_td = partido_element.find_next('td')
                partido = partido_td.get_text(strip=True)

                # Buscar el año dentro de los paréntesis
                anio_inicio_element = partido_td.find('span', {'style': 'font-size:85%;'})
                if anio_inicio_element:
                    anio_inicio = anio_inicio_element.get_text(strip=True).strip('()')
                else:
                    anio_inicio = "No disponible"

                resultados.append({"Nombre": nombre, "Partido político": partido, "Año de inicio": anio_inicio})
            else:
                resultados.append({"Nombre": nombre, "Partido político": "No disponible", "Año de inicio": "No disponible"})
        else:
            resultados.append({"Nombre": nombre, "Partido político": "No disponible", "Año de inicio": "No disponible"})

    # Guardar los resultados en un archivo CSV
    with open('resultados_gobernadores.csv', 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ["Nombre", "Partido político", "Año de inicio"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(resultados)

    # Cerrar el navegador
    driver.quit()
partidos_scraper()

Mtra. María Teresa Jiménez Esquivel 01/10/2022 30/09/2028
C.P. Martín Orozco Sandoval 01/12/2016 30/09/2022
Ing. Carlos Lozano de la Torre 01/12/2010 30/11/2016
Ing. Luis Armando Reynoso Femat 01/12/2004 30/11/2010
Lic. Juan José León Rubio 24/08/2004 30/11/2004
Sr. Felipe González González 01/12/1998 23/08/2004
Mtra. Marina del Pilar Ávila Olmeda 01/11/2021 31/10/2027
Ing. Jaime Bonilla Valdez 01/11/2019 31/10/2021
Lic. Francisco Arturo Vega de Lamadrid 01/11/2013 31/10/2019
Lic. José Guadalupe Osuna Millán 01/11/2007 31/10/2013
Lic. Eugenio Elorduy Walther 01/11/2001 31/10/2007
Mtro. Víctor Manuel Castro Cosío 10/09/2021 09/09/2027
Mtro. Carlos Mendoza Davis 10/09/2015 09/09/2021
C. Marcos Alberto Covarrubias Villaseñor 05/04/2011 09/09/2015
Ing. Narciso Agúndez Montaño 05/04/2005 04/04/2011
Lic. Leonel Efraín Cota Montaño 05/04/1999 04/04/2005
Mtra. Layda Elena Sansores San Román 16/09/2021 15/09/2027
Lic. Carlos Miguel Aysa González 13/06/2019 15/09/2021
Lic. Rafael Alejandro Moren

In [30]:
df_gob = pd.read_csv("datos.csv")
df_gob['Nombre'] = df_gob['Nombre'].str.replace(r'\w+\.', '', regex=True)
df_gob['Nombre'] = df_gob['Nombre'].str.replace(r'\s', '', regex=True)
#df_gob = df_gob.drop_duplicates(subset=['Nombre'])
df_gob

,Nombre,Fecha de Inicio,Fecha de Fin,Estado
0,MaríaTeresaJiménezEsquivel,01/10/2022,30/09/2028,Aguascalientes
1,MartínOrozcoSandoval,01/12/2016,30/09/2022,Aguascalientes
2,CarlosLozanodelaTorre,01/12/2010,30/11/2016,Aguascalientes
3,LuisArmandoReynosoFemat,01/12/2004,30/11/2010,Aguascalientes
4,JuanJoséLeónRubio,24/08/2004,30/11/2004,Aguascalientes
...,...,...,...,...
170,DavidMonrealÁvila,12/09/2021,11/09/2027,Zacatecas
171,AlejandroTelloCristerna,12/09/2016,11/09/2021,Zacatecas
172,MiguelAlejandroAlonsoReyes,12/09/2010,11/09/2016,Zacatecas
173,AmaliaGarcíaMedina,12/09/2004,11/09/2010,Zacatecas


In [27]:
df_pp = pd.read_csv("resultados_gobernadores.csv")
df_pp['Nombre'] = df_pp['Nombre'].str.replace(r'\s', '', regex=True)
df_pp

,Nombre,Partido político,Año de inicio
0,MaríaTeresaJiménezEsquivel,Partido Acción Nacional(desde 2002),desde 2002
1,MartínOrozcoSandoval,Partido Acción Nacional,No disponible
2,CarlosLozanodelaTorre,Partido Revolucionario Institucional,No disponible
3,LuisArmandoReynosoFemat,Fuerza por México(2021)[1]​Partido Acción Naci...,No disponible
4,JuanJoséLeónRubio,Partido Acción Nacional,No disponible
...,...,...,...
140,PatricioJoséPatrónLaviada,Partido Acción Nacional,No disponible
141,DavidMonrealÁvila,Movimiento Regeneración Nacional(desde 2014)Pa...,desde 2014
142,AlejandroTelloCristerna,Partido Revolucionario Institucional,No disponible
143,MiguelAlejandroAlonsoReyes,Partido Revolucionario Institucional(1993-1998...,"1993-1998, desde 2009"


In [11]:
filtrado_ejemplo = df_pp.loc[df_pp['Año de inicio']!='No disponible']
filtrado_ejemplo

,Nombre,Partido político,Año de inicio
0,María Teresa Jiménez Esquivel,Partido Acción Nacional(desde 2002),desde 2002
18,Claudia Sheinbaum Pardo,Movimiento Regeneración Nacional(desde 2014)Pa...,desde 2014
54,Julio Ramón Menchaca Salazar,Movimiento Regeneración Nacional(desde 2017)In...,desde 2017
59,Enrique Alfaro Ramírez,Sin Partido(desde 2012)[2]​[3]​Partido de la R...,desde 2012
61,Emilio González Márquez,Partido Acción Nacional(1992-2015; desde 2021)...,1992-2015; desde 2021
67,Fausto Vallejo Figueroa,Partido Encuentro Social(2018)Partido Revoluci...,2018
68,Fausto Vallejo Figueroa,Partido Encuentro Social(2018)Partido Revoluci...,2018
69,Leonel Godoy Rangel,Movimiento Regeneración Nacional(desde 2017)Pa...,desde 2017
73,Sergio Alberto Estrada Cajigal Ramírez,Fuerza por México(desde 2021)[1]​Partido Acció...,desde 2021
78,Antonio Echevarría Domínguez,Partido Acción Nacional(desde 1998)Partido Rev...,desde 1998


In [43]:
df_merge = pd.merge(df_pp, df_gob, on='Nombre')

partidos = []
years = []

pattern = r'(\w+)\((\d{4})'
matches = df_merge['Partido político'].str.findall(pattern)

for match in matches:
    if match:
        m = match[0]
        partidos.append(m[0])
        year_partido = [int(year) for year in m[1].split('-')]
        year_max = max(year_partido)
        years.append(year_max)

data = {'Partido': partidos, 'Año': years}
df_temporal = pd.DataFrame(data)

df_merge = pd.concat([df_merge, df_temporal], axis=1)  # Fusionar con el DataFrame original

print(df_merge)



                         Nombre  \
0    MaríaTeresaJiménezEsquivel   
1          MartínOrozcoSandoval   
2         CarlosLozanodelaTorre   
3       LuisArmandoReynosoFemat   
4             JuanJoséLeónRubio   
..                          ...   
146   PatricioJoséPatrónLaviada   
147           DavidMonrealÁvila   
148     AlejandroTelloCristerna   
149  MiguelAlejandroAlonsoReyes   
150         RicardoMonrealÁvila   

                                      Partido político          Año de inicio  \
0                  Partido Acción Nacional(desde 2002)             desde 2002   
1                              Partido Acción Nacional          No disponible   
2                 Partido Revolucionario Institucional          No disponible   
3    Fuerza por México(2021)[1]​Partido Acción Naci...          No disponible   
4                              Partido Acción Nacional          No disponible   
..                                                 ...                    ...   
146         

## Scraper SHF 

In [57]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import shutil
from datetime import datetime
from selenium.webdriver.common.action_chains import ActionChains
import requests
import PyPDF2
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

def ift_scraper():
    chrome_options = Options()
    #chrome_options.add_argument('--headless')
    #chrome_options.add_argument('--disable-dev-shm-usage')   
    chrome_options.add_experimental_option("prefs", {
    "download.prompt_for_download": False,  # Desactiva la ventana emergente de descarga
    "download.directory_upgrade": True,
    "safebrowsing.enabled": False,  # Desactiva la verificación de seguridad de descargas
    })
    
    #remote_webdriver = 'remote_chromedriver'
    driver = webdriver.Chrome( options=chrome_options)
    # Scraping part
    url = "https://www.gob.mx/shf/documentos/indice-shf-de-precios-de-la-vivienda-en-mexico-2021-a-2025?state=published"
    # Configuración para ingresar al explorador
    #driver = webdriver.Chrome(options = chrome_options)
    driver.get(url)
    wait= WebDriverWait(driver, 10)
        
        # Encuentra todos los elementos li con la clase "clearfix documents"
    document_elements = driver.find_elements(By.CSS_SELECTOR, "li.clearfix.documents")
        
        # Verifica si hay al menos un elemento
    if document_elements:
            # Encuentra el primer elemento de la lista
        first_document = document_elements[0]
            
            # Encuentra botón de descarga
        pdf_link = first_document.find_element(By.CSS_SELECTOR, "a[href$='.pdf']")
            
            # Obtiene el enlace del atributo href
        pdf_url = pdf_link.get_attribute("href")
            
            # Abre el enlace en una nueva ventana
        driver.execute_script("window.open('" + pdf_url + "', '_blanck');")
            
            # Cambia el enfoque a la nueva ventana
        driver.switch_to.window(driver.window_handles[-1])
            
            # Obtener URL actual
        url_actual = driver.current_url
        print("URL actual", url_actual)
            
            # Ruta destino
        ruta_destino = "SHF.pdf"
            
            # Realiza la solicitud HTTP para descargar el archivo
        response=requests.get(url_actual)
            
            # Verifica si la descarga fue exitosa (código de estado 200)
        if response.status_code == 200:
                # Abre un archivo en modo binario y guarda el contenido del PDF
            with open(ruta_destino, 'wb') as pdf_file:
                pdf_file.write(response.content)
    driver.quit()


    def pdf_convert():
        pdf_file = "SHF.pdf"
            
        # Abre el archivo PDF
        pdf_file = open(pdf_file, 'rb')

        # Crea un objeto PdfFileReader para leer el PDF
        pdf_reader = PyPDF2.PdfReader(pdf_file)

        # Inicializa una lista para almacenar el texto de cada página
        page_texts = []

        # Itera a través de las páginas y extrae el texto
        for page in pdf_reader.pages:
            page_texts.append(page.extract_text())

        # Cierra el archivo PDF
        pdf_file.close()

        # Combina el texto de todas las páginas en un solo texto
        all_text = '\n'.join(page_texts)

        # Divide el texto en líneas
        lines = all_text.split('\n')

        # Crea un DataFrame a partir de las líneas
        df = pd.DataFrame(lines, columns=['Texto'])

        ruta_csv = "SHF.csv"    
        df.to_csv(ruta_csv, index=False)
        return df
    pdf_convert()

    def extractor_tablas():
        df = pd.read_csv("SHF.csv")
        df = df.drop(df.index[48:])
        # Reajusta el índice para que comience desde 0
        df = df.reset_index(drop=True)
        # Elimina la primera fila, que contiene el encabezado, y guarda el resultado en un nuevo DataFrame
        df_sin_encabezado = df.iloc[1:].copy()

        # Extrae la columna 'Valores'
        df_sin_encabezado['Valores'] = df_sin_encabezado['Texto'].str.extract(r'(\d[\d. \s]+)')
        # Dividir la columna 'Valores' en múltiples columnas usando espacios en blanco como separadores
        columnas_valores = df_sin_encabezado['Valores'].str.split(expand=True)
        # Eliminar filas que contienen valores NaN (en este caso, la fila que contiene "Na")
        columnas_valores = columnas_valores.dropna()

        df_sin_encabezado['Estado'] = (df_sin_encabezado['Texto'].str.extract('([^0-9]+)'))
        # Combina la columna 'Estado' de df_sin_encabezado con columnas_valores
        columnas_valores.insert(0, 'Estado', df_sin_encabezado['Estado'].values)
        # Acceder a la segunda columna por posición
        segunda_columna = columnas_valores.iloc[:, 1]
        # Lista de índices de filas a modificar 
        filas_a_modificar = [0, 33, 34]

        for fila_idx in filas_a_modificar:
            # Modificar los valores de la segunda columna
            segunda_columna.iloc[fila_idx] = str(segunda_columna.iloc[fila_idx])[1:]

        columnas_valores = columnas_valores.rename(columns={columnas_valores.columns[0]: 'Estado'})


        #Obtener la longitud 
        num_columnas = len(columnas_valores.columns)
        # Crear una lista de años desde 2015 hasta 2024
        years = range(2015, 2025)
        # Crear una lista de números romanos del I al IV
        romanos = ['I', 'II', 'III', 'IV']
        # Crea una lista de nombres de columna basados en el patrón deseado
        nombres_columnas = ["Estado"] + [f"{anno}-{romano}" for anno in years for romano in romanos]
        # Asegurarse de que la lista de nombres de columna tenga la misma longitud que la tabla
        nombres_columnas = nombres_columnas[:num_columnas]
        columnas_valores.columns = nombres_columnas
        # Guardar el DataFrame modificado en un archivo CSV
        columnas_valores.to_csv("SHF_extract.csv", index=False)
    extractor_tablas()
ift_scraper()

URL actual https://www.gob.mx/cms/uploads/attachment/file/848146/_ndice_SHF_T2_2023.pdf


## Scraper Desempleo

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import shutil
from datetime import datetime
from selenium.webdriver.common.action_chains import ActionChains
import requests
import PyPDF2
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import os



def INEGI_desempleo_scraper():

    # Get the directory of the current script
    script_directory = os.path.dirname(os.path.realpath(__file__))

    # Define the download directory relative to the script's location
    download_directory = os.path.join(script_directory, 'downloads')

    # Ensure the download directory exists
    if not os.path.exists(download_directory):
        os.makedirs(download_directory)

    url = "https://www.inegi.org.mx/app/tabulados/default.html?nc=624"
    
    chrome_options = Options()
    chrome_options.add_experimental_option("prefs", {
        "download.prompt_for_download": False,
        "download.default_directory": download_directory,  # Set to the dynamically determined path
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True,
        "profile.default_content_settings.popups": 0,
        "profile.default_content_setting_values.automatic_downloads": 2,
    })

    # Configuración para ingresar al explorador
    #driver = webdriver.Chrome(options = chrome_options)
    remote_webdriver = 'remote_chromedriver'
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    wait= WebDriverWait(driver, 10)
    button = wait.until(EC.element_to_be_clickable((By.ID, "aCsv")))
    button.click()
    time.sleep(20)
    driver.quit()
    
    def data_transformation():
        df = pd.read_csv("Tabulado.csv")
        # Encuentra los índices donde comienzan los años en la columna 'Periodo'
        indices_anios = df[df['Periodo'].str.isnumeric()].index
        nuevo_df = df.T.iloc[1:].reset_index(drop=True)
        nuevo_df = nuevo_df.dropna(axis =1)
        num_columnas = len(nuevo_df.columns)
            
        years = range(2020,2025)
        # Lista de números romanos
        romanos = ['I', 'II', 'III', 'IV']
        # Crea una lista de nombres de columnas basados en el patrón
        nombres_columnas = [f"{anno}-{romano}" for anno in years for romano in romanos]
        nombres_columnas = nombres_columnas[:num_columnas]
        #Asigna los nombres de las columnas al nuevo DataFrame
        nuevo_df.columns = nombres_columnas 
        # Agrega una columna 'Estado' al nuevo DataFrame
        nuevo_df.insert(0, 'Estado', df.columns[1:])

        # Restablece el índice
        nuevo_df.reset_index(drop=True, inplace=True)
        nuevo_df.to_csv("Tabulado.csv", index = False)
    data_transformation()
INEGI_desempleo_scraper()

NameError: name '__file__' is not defined

## Scraper Clima

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import pandas as pd
import json
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import warnings
from bs4 import BeautifulSoup
import json
import csv
from urllib.parse import urlparse
from selenium.common.exceptions import WebDriverException

warnings.filterwarnings('ignore')

def obtener_nombre_estado(url):
    parsed_url = urlparse(url)
    path = parsed_url.path
    parts = path.split("/")
    if len(parts) >= 3:
        estado = parts[2].replace("-", " ")
        # Tratar la excepción de "distrito-federal"
        if "estado-de" in estado:
            estado = estado.replace("estado-de-", "")
        elif "distrito-federal" in estado:
            estado = "Ciudad de México"
        return estado
    return None


def clima_scraper():
    max_attempts = 4
    attempts = 0

    while attempts < max_attempts:
        datos_climaticos = []
        try:

            urls =  [
            "https://www.clima.com/mexico/estado-de-aguascalientes/aguascalientes",
            "https://www.clima.com/mexico/estado-de-baja-california/mexicali",
            "https://www.clima.com/mexico/estado-de-baja-california-sur/la-paz",
            "https://www.clima.com/mexico/estado-de-campeche/campeche",
            "https://www.clima.com/mexico/estado-de-chiapas/tuxtla-gutierrez",
            "https://www.clima.com/mexico/estado-de-chihuahua/chihuahua",
            "https://www.clima.com/mexico/distrito-federal/mexico",
            "https://www.clima.com/mexico/estado-de-coahuila-de-zaragoza/saltillo",
            "https://www.clima.com/mexico/estado-de-colima/colima",
            "https://www.clima.com/mexico/estado-de-durango/victoria-de-durango",
            "https://www.clima.com/mexico/estado-de-guanajuato/guanajuato",
            "https://www.clima.com/mexico/estado-de-guerrero/chilpancingo-de-los-bravos",
            "https://www.clima.com/mexico/estado-de-hidalgo/pachuca-de-soto",
            "https://www.clima.com/mexico/estado-de-jalisco/guadalajara",
            "https://www.clima.com/mexico/estado-de-mexico/ecatepec",
            "https://www.clima.com/mexico/estado-de-michoacan-de-ocampo/morelia",
            "https://www.clima.com/mexico/estado-de-morelos/cuernavaca",
            "https://www.clima.com/mexico/estado-de-nayarit/tepic",
            "https://www.clima.com/mexico/estado-de-nuevo-leon/monterrey",
            "https://www.clima.com/mexico/estado-de-oaxaca/oaxaca-de-juarez",
            "https://www.clima.com/mexico/estado-de-puebla/puebla-de-zaragoza",
            "https://www.clima.com/mexico/estado-de-queretaro-de-arteaga/queretaro",
            "https://www.clima.com/mexico/estado-de-quintana-roo/cancun-2",
            "https://www.clima.com/mexico/estado-de-san-luis-potosi/san-luis-potosi",
            "https://www.clima.com/mexico/estado-de-sinaloa/culiacan",
            "https://www.clima.com/mexico/estado-de-sonora/hermosillo",
            "https://www.clima.com/mexico/estado-de-tabasco/villahermosa",
            "https://www.clima.com/mexico/estado-de-tamaulipas/reynosa",
            "https://www.clima.com/mexico/estado-de-tlaxcala/tlaxcala",
            "https://www.clima.com/mexico/estado-de-veracruz-llave/veracruz",
            "https://www.clima.com/mexico/estado-de-yucatan/merida",
            "https://www.clima.com/mexico/estado-de-zacatecas/zacatecas"
        ]
        
            
            chrome_options = Options()
            driver = webdriver.Chrome(options=chrome_options)

            for url in urls:
                driver.get(url)
                wait = WebDriverWait(driver, 10)

                container= wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "table")))
                container.location_once_scrolled_into_view

                desplazamiento_vertical = 400  # Cambia esto a la cantidad de píxeles que desees desplazarte

                # Realiza el desplazamiento
                driver.execute_script(f"window.scrollBy(0, {desplazamiento_vertical});")

                time.sleep(10)

                #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                page_soruce = driver.page_source
                soup = BeautifulSoup(page_soruce, "html.parser")
                # Encontrar el elemento <canvas>
                canvas = soup.find('canvas', class_='chart gtm-climateAverage-chart')

                # Obtener el valor del atributo 'data-months'
                data_months = canvas['data-months']

                
                months_data = json.loads(data_months)
                # Obtener el nombre del estado
                nombre_estado = obtener_nombre_estado(url)

                for mes in months_data:
                    mes['Estado'] = nombre_estado # Agregar la nueva columna con el nombre del estado

                datos_climaticos.extend(months_data)

                # Crear un archivo CSV y escribir los datos
                with open('datos_climaticos.csv', mode='w', newline='') as file:
                    fieldnames = list(months_data[0].keys())  # Obtener los nombres de las columnas
                    #fieldnames.append('Estado')  # Agregar la nueva columna 'Estado'
                    writer = csv.DictWriter(file, fieldnames=fieldnames)
                    writer.writeheader()
                    writer.writerows(datos_climaticos)
            break
        except WebDriverException as e:
            print(f"Se produjo una excepción: {e}")
            attempts += 1
            print(f"Reintentando (Intento {attempts}/{max_attempts})...")
            driver.quit()
            continue
    if attempts == max_attempts:
        print(f"Se alcanzó el número máximo de intentos")
        driver.quit()

clima_scraper()

Se produjo una excepción: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=121.0.6167.187)
Stacktrace:
	GetHandleVerifier [0x00007FF77CC87012+3522402]
	(No symbol) [0x00007FF77C8A8352]
	(No symbol) [0x00007FF77C755ABB]
	(No symbol) [0x00007FF77C73287C]
	(No symbol) [0x00007FF77C7C5D97]
	(No symbol) [0x00007FF77C7DB3CF]
	(No symbol) [0x00007FF77C7BEE03]
	(No symbol) [0x00007FF77C78F4D4]
	(No symbol) [0x00007FF77C7905F1]
	GetHandleVerifier [0x00007FF77CCB9B9D+3730157]
	GetHandleVerifier [0x00007FF77CD0F02D+4079485]
	GetHandleVerifier [0x00007FF77CD075D3+4048163]
	GetHandleVerifier [0x00007FF77C9DA649+718233]
	(No symbol) [0x00007FF77C8B4A3F]
	(No symbol) [0x00007FF77C8AFA94]
	(No symbol) [0x00007FF77C8AFBC2]
	(No symbol) [0x00007FF77C89F2E4]
	BaseThreadInitThunk [0x00007FFFF8D4257D+29]
	RtlUserThreadStart [0x00007FFFFA3AAA58+40]

Reintentando (Intento 1/4)...


AttributeError: 'NoneType' object has no attribute 'is_displayed'

In [ ]:
df_gob = pd.read_csv("datos.csv")
df_gob['Nombre'] = df_gob['Nombre'].str.replace(r'\w+\.', '', regex=True)
df_gob['Nombre'] = df_gob['Nombre'].str.replace(r'\s', '', regex=True)
#df_gob = df_gob.drop_duplicates(subset=['Nombre'])
df_gob

## Union de datos

### Acomodo gobernador

In [18]:
df_gob = pd.read_csv("datos.csv")
df_gob['Nombre'] = df_gob['Nombre'].str.replace(r'\w+\.', '', regex=True)
df_gob['Nombre'] = df_gob['Nombre'].str.replace(r'\s', '', regex=True)
df_gob['Estado'] = df_gob['Estado'].str.lower()
df_gob

,Nombre,Fecha de Inicio,Fecha de Fin,Estado
0,MaríaTeresaJiménezEsquivel,01/10/2022,30/09/2028,aguascalientes
1,MartínOrozcoSandoval,01/12/2016,30/09/2022,aguascalientes
2,CarlosLozanodelaTorre,01/12/2010,30/11/2016,aguascalientes
3,LuisArmandoReynosoFemat,01/12/2004,30/11/2010,aguascalientes
4,JuanJoséLeónRubio,24/08/2004,30/11/2004,aguascalientes
...,...,...,...,...
170,DavidMonrealÁvila,12/09/2021,11/09/2027,zacatecas
171,AlejandroTelloCristerna,12/09/2016,11/09/2021,zacatecas
172,MiguelAlejandroAlonsoReyes,12/09/2010,11/09/2016,zacatecas
173,AmaliaGarcíaMedina,12/09/2004,11/09/2010,zacatecas


### Acomodo por partido Político por gobernador

In [28]:
df_pp = pd.read_csv("resultados_gobernadores.csv")
df_pp

,Nombre,Partido político,Año de inicio
0,María Teresa Jiménez Esquivel,Partido Acción Nacional(desde 2002),desde 2002
1,Martín Orozco Sandoval,Partido Acción Nacional,No disponible
2,Carlos Lozano de la Torre,Partido Revolucionario Institucional,No disponible
3,Luis Armando Reynoso Femat,Fuerza por México(2021)[1]​Partido Acción Naci...,No disponible
4,Juan José León Rubio,Partido Acción Nacional,No disponible
...,...,...,...
140,Patricio José Patrón Laviada,Partido Acción Nacional,No disponible
141,David Monreal Ávila,Movimiento Regeneración Nacional(desde 2014)Pa...,desde 2014
142,Alejandro Tello Cristerna,Partido Revolucionario Institucional,No disponible
143,Miguel Alejandro Alonso Reyes,Partido Revolucionario Institucional(1993-1998...,"1993-1998, desde 2009"


In [38]:
df_pp = pd.read_csv("resultados_gobernadores.csv")
df_pp['Nombre'] = df_pp['Nombre'].str.replace(r'\s', '', regex=True)

def split_parties(row):
    party = None
    year = None

    match = re.search(r'(.+?)\(([^)]+)\)', row)
    if match:
        party = match.group(1)
        years_in_match = re.findall(r'\d{4}', match.group(2))
        if years_in_match:
            year = max(int(year) for year in years_in_match)
    
    return party, year


# Aplicar la función a cada fila
split_data = df_pp['Partido político'].apply(split_parties)
df_pp['Partido'] = split_data.apply(lambda x: x[0])
df_pp['Año'] = split_data.apply(lambda x: x[1])
df_pp['Partido'].fillna(df_pp['Partido político'], inplace=True)

df_pp= df_pp[['Nombre', 'Partido']]
print(df_pp)


                         Nombre                               Partido
0    MaríaTeresaJiménezEsquivel               Partido Acción Nacional
1          MartínOrozcoSandoval               Partido Acción Nacional
2         CarlosLozanodelaTorre  Partido Revolucionario Institucional
3       LuisArmandoReynosoFemat                     Fuerza por México
4             JuanJoséLeónRubio               Partido Acción Nacional
..                          ...                                   ...
140   PatricioJoséPatrónLaviada               Partido Acción Nacional
141           DavidMonrealÁvila      Movimiento Regeneración Nacional
142     AlejandroTelloCristerna  Partido Revolucionario Institucional
143  MiguelAlejandroAlonsoReyes  Partido Revolucionario Institucional
144         RicardoMonrealÁvila      Movimiento Regeneración Nacional

[145 rows x 2 columns]


### Merge entre gobernadores y partidos políticos

In [41]:
df_merge = pd.merge(df_pp, df_gob, on='Nombre')
df_merge


,Nombre,Partido,Fecha de Inicio,Fecha de Fin,Estado
0,MaríaTeresaJiménezEsquivel,Partido Acción Nacional,01/10/2022,30/09/2028,aguascalientes
1,MartínOrozcoSandoval,Partido Acción Nacional,01/12/2016,30/09/2022,aguascalientes
2,CarlosLozanodelaTorre,Partido Revolucionario Institucional,01/12/2010,30/11/2016,aguascalientes
3,LuisArmandoReynosoFemat,Fuerza por México,01/12/2004,30/11/2010,aguascalientes
4,JuanJoséLeónRubio,Partido Acción Nacional,24/08/2004,30/11/2004,aguascalientes
...,...,...,...,...,...
146,PatricioJoséPatrónLaviada,Partido Acción Nacional,01/08/2001,31/07/2007,yucatán
147,DavidMonrealÁvila,Movimiento Regeneración Nacional,12/09/2021,11/09/2027,zacatecas
148,AlejandroTelloCristerna,Partido Revolucionario Institucional,12/09/2016,11/09/2021,zacatecas
149,MiguelAlejandroAlonsoReyes,Partido Revolucionario Institucional,12/09/2010,11/09/2016,zacatecas


### Acomodo SHF

In [64]:
df_shf = pd.read_csv('SHF_extract.csv')
df_shf = df_shf.drop(df_shf.index[33:])
df_shf = df_shf.melt(id_vars=['Estado'], var_name = 'Trimestre', value_name='SHF')
# Usar str.contains para filtrar los registros que contienen "2023"
df_shf = df_shf[df_shf['Trimestre'].str.contains('2023')]
df_shf['Estado'] = df_shf['Estado'].str.lower()
df_shf

,Estado,Trimestre,SHF
1056,nacional,2023-I,157.6
1057,aguascalientes,2023-I,154.1
1058,baja california,2023-I,172.2
1059,baja california sur,2023-I,182.3
1060,campeche,2023-I,166.5
...,...,...,...
1117,tamaulipas,2023-II,154.4
1118,tlaxcala,2023-II,144.5
1119,veracruz,2023-II,156.3
1120,yucatán,2023-II,163.6


### Acomodo desempleo

In [100]:
from unidecode import unidecode
df_desemp = pd.read_csv('Tabulado.csv')
df_desemp = df_desemp.melt(id_vars=['Estado'], var_name='Trimestre', value_name='DP')

renombre = {
    'Coahuila de Zaragoza' : 'Coahuila',
    'Distrito Federal' : 'Ciudad de México',
    'Michoacán de Ocampo' : 'Michoacán',
    'Veracruz de Ignacio de la Llave':'Veracruz'
}

df_desemp['Estado']=df_desemp['Estado'].replace(renombre)
df_desemp['Estado'] = df_desemp['Estado'].str.lower() 
df_desemp['Estado'] = df_desemp['Estado'].apply(unidecode)
df_desemp


,Estado,Trimestre,DP
0,aguascalientes,2020-I,3.350091
1,baja california,2020-I,2.167978
2,baja california sur,2020-I,3.837031
3,campeche,2020-I,2.782535
4,coahuila,2020-I,4.767192
...,...,...,...
443,tamaulipas,2023-II,3.151739
444,tlaxcala,2023-II,3.405865
445,veracruz,2023-II,2.138434
446,yucatan,2023-II,1.576813


### Acomodo datos climáticos

In [96]:
df_clima = pd.read_csv('datos_climaticos.csv')
df_clima['Estado'] = df_clima['Estado'].str.replace(r'^estado de ', '', regex=True)
renombre = {
    'coahuila de zaragoza' : 'coahuila',
    'distrito federal' : 'ciudad de mexico',
    'michoacan de ocampo' : 'michoacan',
    'veracruz llave':'veracruz',
    'queretaro de arteaga': 'queretaro',
}

df_clima['Estado']=df_clima['Estado'].replace(renombre)
df_clima


,monthIndex,monthDays,daysOfPrecip,rain,temp,tempMax,tempMin,windSpeed,name,Estado
0,1,31,3,18,12,21,4,9,Enero,aguascalientes
1,2,28,2,11,14,24,6,10,Febrero,aguascalientes
2,3,31,3,23,17,26,8,10,Marzo,aguascalientes
3,4,30,5,16,19,29,10,10,Abril,aguascalientes
4,5,31,8,35,21,30,13,9,Mayo,aguascalientes
...,...,...,...,...,...,...,...,...,...,...
379,8,31,20,136,16,22,11,15,Agosto,zacatecas
380,9,30,18,142,15,21,10,15,Septiembre,zacatecas
381,10,31,10,60,14,20,8,16,Octubre,zacatecas
382,11,30,6,30,11,18,5,17,Noviembre,zacatecas


In [102]:
# Mapeo de meses a trimestres
meses_a_trimestres = {
    'Enero': 'I',
    'Febrero': 'I',
    'Marzo': 'I',
    'Abril': 'II',
    'Mayo': 'II',
    'Junio': 'II',
    'Julio': 'III',
    'Agosto': 'III',
    'Septiembre': 'III',
    'Octubre': 'IV',
    'Noviembre': 'IV',
    'Diciembre': 'IV'
}

# Agregar una columna de Trimestre al DataFrame de meses
df_clima['Trimestre'] = df_clima['name'].map(meses_a_trimestres)

# Fusionar DataFrames en función del Estado y el Trimestre
df_final = df_clima.merge(df_desemp, on=['Estado', 'Trimestre'])

# Mostrar el DataFrame resultante
print(df_final)

Empty DataFrame
Columns: [monthIndex, monthDays, daysOfPrecip, rain, temp, tempMax, tempMin, windSpeed, name, Estado, Trimestre, DP]
Index: []


## Scraper REPEP

In [16]:
telefonos = ['5514602894', '5557780290', '3316234252']
def repep_scraper():
    repep = []
    for telefono in telefonos :
        chrome_options = Options()
        # Configuraciones adicionales omitidas para brevedad

        driver = webdriver.Chrome(options=chrome_options)
        url = "https://repep.profeco.gob.mx/Solicitudnumero.jsp"
        driver.get(url)
        wait = WebDriverWait(driver, 10)
            
        # Encuentra el elemento y realiza las acciones
        input_repep = driver.find_element(By.ID, "telefono")
        input_repep.click()  # Primero haz click en el elemento
        input_repep.send_keys(telefono)  # Luego envía las teclas
        input_repep.submit()  # Finalmente, realiza el envío del formulario
        time.sleep(5)
        try : 
            #Esperar por la vetnana emergente durante 10 minutos
            WebDriverWait(driver, 10).until(EC.alert_is_present())
            #Cambiar el enfoque a la vetnana emergente
            alert = driver.switch_to.alert
            # Obtener el texto de la ventana emergente
            alert_text = alert.text
            print(f'Alerta encontrada:{alert_text}')
            alert.accept()
        except TimeoutException:
            # Si no se encuentra ninguna ventana emergente en 10 segundos
            print("No se encontró la ventana emergente.")
            repep.append(telefono)
            print(repep)
    
        driver.quit()
    repep_df = pd.DataFrame(repep, columns=['Telefono'])
    repep_df.to_csv('repep.csv', index=False)
    
repep_scraper()



Alerta encontrada:El número telefónico no está registrado
Alerta encontrada:El número telefónico no está registrado
No se encontró la ventana emergente.
['3316234252']


AttributeError: 'list' object has no attribute 'to_csv'

# Scrapers para Calidad


## 1. Nuxiba 

In [253]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from dotenv import load_dotenv, find_dotenv
import pandas as pd
import shutil
from datetime import datetime
from selenium.webdriver.common.action_chains import ActionChains
import requests
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities


def nuxiba_calls():
    url = 'https://cwkolob182.nuxiba.com/adminkolob/?softphone=WebRTC#/'
    load_dotenv(find_dotenv())
    
    chrome_options = Options()
    chrome_options.add_experimental_option("prefs", {
    "download.prompt_for_download": False,  # Desactiva la ventana emergente de descarga
    "download.directory_upgrade": True,
    "safebrowsing.enabled": False,  # Desactiva la verificación de seguridad de descargas
    #"download.default_directory": "C:/Users/DSTHREE/Documents/GITHUB/airflow/dags/utils",
})
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    
    wait= WebDriverWait(driver, 20)
    userid = os.getenv("USER_NUXIBA")
    user_entry = wait.until(EC.presence_of_element_located((By.ID, "logintxtUsername")))
    user_entry.send_keys(userid)
    password = os.getenv("PASSWORD_NUXIBA")
    password_entry = driver.find_element(By.CLASS_NAME, "inputPassword.form-control")
    password_entry.send_keys(password)
    password_entry.send_keys(Keys.ENTER)
    llamadas = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="finder-sidebar"]/i')))
    llamadas.click()
    time.sleep(10)



    driver.execute_script("document.body.style.zoom='50%'")
    time.sleep(2)



    data = []
    previous_num_of_rows = 0
    while True:
        table_rows = driver.find_elements(By.CSS_SELECTOR, '.sbft-rows tr')

        if len(table_rows) == previous_num_of_rows:  # No hay más registros nuevos
            break
        

        for row in table_rows[previous_num_of_rows:]:
            # ... [Tu código para extraer información y añadir a la lista `data`]
            # 1. Extraer información de la fila y guardarla en el diccionario `info`
            fecha = row.find_element(By.XPATH, './td[5]').text
            id_val = row.find_element(By.XPATH, './td[6]').text
            campana = row.find_element(By.XPATH, './td[7]').text
            canal = row.find_element(By.XPATH, './td[8]').text
            nombre_usuario = row.find_element(By.XPATH, './td[9]').text
            nombre_agente = row.find_element(By.XPATH, './td[10]').text
            telefono = row.find_element(By.XPATH, './td[11]').text
            duracion = row.find_element(By.XPATH, './td[12]').text
            calificacion = row.find_element(By.XPATH, './td[13]').text

            info = {
                "Fecha": fecha,
                "ID": id_val,
                "Campaña": campana,
                "Canal": canal,
                "Nombre de Usuario": nombre_usuario,
                "Nombre de Agente": nombre_agente,
                "Teléfono": telefono,
                "Duración": duracion,
                "Calificación": calificacion
            }

            data.append(info)

            element = wait.until(EC.presence_of_element_located((By.ID, "gcl-icon.checkbox"))) 
            if element.is_displayed():
                print("El elemento está visible")
            else:
                print("El elemento no está visible")


            display = element.value_of_css_property("display")
            visibility = element.value_of_css_property("visibility")

            if display == "none":
                print("El elemento tiene 'display: none'")
            if visibility == "hidden":
                print("El elemento tiene 'visibility: hidden'")

            try:
                element.click()
            except Exception as e:
                clicked_element = driver.execute_script("return document.elementFromPoint(arguments[0], arguments[1]);", *element.location.values())
                print(f"En lugar de tu elemento, el elemento {clicked_element.tag_name} con la clase {clicked_element.get_attribute('class')} está recibiendo el clic.")


            #checkbox.click()

            # Espera explícita hasta que el botón esté interactuable
            download = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'gcl-btn.tertiary ')))
            download.click()
            

        # Actualiza el contador
        previous_num_of_rows = len(table_rows)

        # Desplázate un poco hacia abajo para obtener más registros (puedes ajustar este valor según tus necesidades)
        driver.execute_script("window.scrollBy(0, 100);")
        time.sleep(2)

    # Después del bucle, procesa la lista `data`
    df = pd.DataFrame(data)
    print(df)

nuxiba_calls()

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF710678EF2+54786]
	(No symbol) [0x00007FF7105E5612]
	(No symbol) [0x00007FF71049A64B]
	(No symbol) [0x00007FF7104DB79C]
	(No symbol) [0x00007FF7104DB91C]
	(No symbol) [0x00007FF710516D87]
	(No symbol) [0x00007FF7104FBEAF]
	(No symbol) [0x00007FF710514D02]
	(No symbol) [0x00007FF7104FBC43]
	(No symbol) [0x00007FF7104D0941]
	(No symbol) [0x00007FF7104D1B84]
	GetHandleVerifier [0x00007FF7109C7F52+3524194]
	GetHandleVerifier [0x00007FF710A1D800+3874576]
	GetHandleVerifier [0x00007FF710A15D7F+3843215]
	GetHandleVerifier [0x00007FF710715086+694166]
	(No symbol) [0x00007FF7105F0A88]
	(No symbol) [0x00007FF7105ECA94]
	(No symbol) [0x00007FF7105ECBC2]
	(No symbol) [0x00007FF7105DCC83]
	BaseThreadInitThunk [0x00007FF94768257D+29]
	RtlUserThreadStart [0x00007FF9482CAA78+40]
